In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import os

# Parameters
img_size = (224, 224)
batch_size = 32
num_classes = 4
epochs = 10
train_dir = 'dataset_new/train'
test_dir = 'dataset_new/test'

class_names = sorted(os.listdir(train_dir))

# Load dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False
)

# Preprocessing function for VGG16 expects inputs preprocessed accordingly
def preprocess(image, label):
    image = preprocess_input(image)
    return image, label

train_ds = train_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

# Build model using pretrained VGG16 as base
base_model = VGG16(include_top=False, weights='imagenet', input_shape=img_size+(3,))
base_model.trainable = False  # Freeze base layers for transfer learning

inputs = Input(shape=img_size+(3,))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

# Train model
history = model.fit(train_ds, validation_data=test_ds, epochs=epochs)

Found 2467 files belonging to 4 classes.
Found 433 files belonging to 4 classes.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │           1,028 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,847,044 (56.64 MB)

 Trainable params: 132,356 (517.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

None
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 159s 2s/step - accuracy: 0.7519 - loss: 1.0637 - val_accuracy: 0.8152 - val_loss: 0.3398
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.8553 - loss: 0.3092 - val_accuracy: 0.8984 - val_loss: 0.2201
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 169s 2s/step - accuracy: 0.8735 - loss: 0.2619 - val_accuracy: 0.9169 - val_loss: 0.2100
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.8995 - loss: 0.2210 - val_accuracy: 0.9099 - val_loss: 0.1770
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.9023 - loss: 0.2054 - val_accuracy: 0.9238 - val_loss: 0.1675
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.9210 - loss: 0.1751 - val_accuracy: 0.9400 - val_loss: 0.1382
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.9392 - loss: 0.1398 - val_accuracy: 0.9423 - val_loss: 0.1391
Epoch 8/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.9380 - loss: 0.1458 - val_accuracy: 0.970

In [3]:
model.save('vgg16_transfer_model.h5')
print("Hyperparameter tuning completed and best model saved as 'best_vgg16_transfer_model.h5'")


Hyperparameter tuning completed and best model saved as 'best_vgg16_transfer_model.h5'
